In [1]:
import pandas as pd 
import numpy as np 
import time 
import gc 

np.random.seed(42)

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model

from keras import optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

import threading
import multiprocessing
from multiprocessing import Pool, cpu_count
from contextlib import closing

from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

import torch

lentrain = 1503424
max_seq_title_description_length = 100
max_words_title_description = 200000

C:\Users\ahega\Anaconda3\envs\Avito18_Kaggle\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def cleanName(text):
    try:
        textProc = text.lower()
        # textProc = " ".join(map(str.strip, re.split('(\d+)',textProc)))
        #regex = re.compile(u'[^[:alpha:]]')
        #textProc = regex.sub(" ", textProc)
        textProc = re.sub('[!@#$_“”¨«»®´·º½¾¿¡§£₤‘’]', '', textProc)
        textProc = " ".join(textProc.split())
        return textProc
    except: 
        return "name error"

In [2]:
'''
train = pd.read_csv('../features/train/seq_title_description_train.csv')
test = pd.read_csv('../features/test/seq_title_description_test.csv')

print(train.shape, test.shape)
df = pd.concat([train, test])
del train, test; gc.collect()

df.head()
'''

"\ntrain = pd.read_csv('../features/train/seq_title_description_train.csv')\ntest = pd.read_csv('../features/test/seq_title_description_test.csv')\n\nprint(train.shape, test.shape)\ndf = pd.concat([train, test])\ndel train, test; gc.collect()\n\ndf.head()\n"

In [15]:
train = pd.read_csv('../input/train.csv', usecols=['description', 'title'])
test = pd.read_csv('../input/test.csv', usecols=['description', 'title'])

df = pd.concat([train, test])

del train, test; gc.collect()

df.shape

(2011862, 2)

In [16]:
df['title_description'] = (df['title']+' '+df['description']).astype(str)
df.drop(['title', 'description'], axis=1, inplace=True)
gc.collect
df.head()

,title_description
0,"Кокоби(кокон для сна) Кокон для сна малыша,пол..."
1,"Стойка для Одежды Стойка для одежды, под вешал..."
2,"Philips bluray В хорошем состоянии, домашний к..."
3,Автокресло Продам кресло от0-25кг
4,"ВАЗ 2110, 2003 Все вопросы по телефону."


In [17]:
print("Start Tokenization.....")
tokenizer = text.Tokenizer(num_words = max_words_title_description)
all_text = np.hstack([df['title_description'].str.lower()])
tokenizer.fit_on_texts(all_text)
del all_text
gc.collect()

Start Tokenization.....


7

In [18]:
df['seq_title_description']= tokenizer.texts_to_sequences(df.title_description.str.lower())
df.drop(['title_description'], axis=1, inplace=True)
df.shape

(2011862, 1)

In [19]:
df.head(20)

,seq_title_description
0,"[131120, 7400, 5, 2627, 7400, 5, 2627, 486, 52..."
1,"[2330, 5, 669, 2330, 5, 669, 51, 10431, 4, 36172]"
2,"[1803, 94810, 1, 29, 10, 2863, 4065, 4, 9699, ..."
3,"[1029, 8, 506, 43306, 11806]"
4,"[258, 2309, 1749, 16, 135, 6, 87]"
5,"[197, 777, 1, 29, 10]"
6,"[3357, 65, 1157, 8062, 8645, 1265, 3357, 20064..."
7,"[16646, 29119, 16646, 1, 29, 10]"
8,"[43, 523, 66, 40470, 26, 540]"
9,"[1013, 407, 8615, 2826, 23714, 6, 281, 97, 164..."


In [21]:
train = df[:lentrain]
test = df[lentrain:]

test.reset_index(drop=True, inplace=True)
print(train.shape)
print(test.shape)

train.to_csv('../features/train/seq_tokenized.csv', index=False)
test.to_csv('../features/test/seq_tokenized.csv', index=False)

del train, test; gc.collect()

(1503424, 1)
(508438, 1)


145

In [23]:
EMBEDDING_DIM1 = 300
EMBEDDING_FILE1 = './wordembeds/cc.ru.300.vec'
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index1 = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE1, encoding='utf-8'))

vocab_size = len(tokenizer.word_index)+2
EMBEDDING_DIM1 = 300# this is from the pretrained vectors
embedding_matrix1 = np.zeros((vocab_size, EMBEDDING_DIM1))
print(embedding_matrix1.shape)
# Creating Embedding matrix 
c = 0 
c1 = 0 
w_Y = []
w_No = []
for word, i in tokenizer.word_index.items():
    if word in embeddings_index1:
        c +=1
        embedding_vector = embeddings_index1[word]
        w_Y.append(word)
    else:
        embedding_vector = None
        w_No.append(word)
        c1 +=1
    if embedding_vector is not None:    
        embedding_matrix1[i] = embedding_vector

print(c,c1, len(w_No), len(w_Y))
print(embedding_matrix1.shape)
del embeddings_index1
gc.collect()

print(" FAST TEXT DONE")

(895302, 300)
307394 587906 587906 307394
(895302, 300)
 FAST TEXT DONE


In [29]:
temp = embedding_matrix1.astype('float32')
np.save('./wordembeds/new_cc_matrix.npy', temp)

In [30]:
EMBEDDING_DIM1 = 300
EMBEDDING_FILE1 = './wordembeds/wiki.ru.vec'
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index1 = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE1, encoding='utf-8'))

vocab_size = len(tokenizer.word_index)+2
EMBEDDING_DIM1 = 300# this is from the pretrained vectors
embedding_matrix1 = np.zeros((vocab_size, EMBEDDING_DIM1))
print(embedding_matrix1.shape)
# Creating Embedding matrix 
c = 0 
c1 = 0 
w_Y = []
w_No = []
for word, i in tokenizer.word_index.items():
    if word in embeddings_index1:
        c +=1
        embedding_vector = embeddings_index1[word]
        w_Y.append(word)
    else:
        embedding_vector = None
        w_No.append(word)
        c1 +=1
    if embedding_vector is not None:    
        embedding_matrix1[i] = embedding_vector

print(c,c1, len(w_No), len(w_Y))
print(embedding_matrix1.shape)
del embeddings_index1
gc.collect()

print(" FAST TEXT DONE")

(895302, 300)
307251 588049 588049 307251
(895302, 300)
 FAST TEXT DONE


In [31]:
temp = embedding_matrix1.astype('float32')
np.save('./wordembeds/new_wiki_matrix.npy', temp)

In [16]:
train_words = pd.read_csv('../features/train/seq_tokenized.csv')
train_words['seq_title_description'] = train_words['seq_title_description'].str.strip('[]').str.replace('\s+', '')
train_words['seq_title_description'] = train_words['seq_title_description'].apply(lambda x: x.split(','))
train_words.head()

,seq_title_description
0,"[131120, 7400, 5, 2627, 7400, 5, 2627, 486, 52..."
1,"[2330, 5, 669, 2330, 5, 669, 51, 10431, 4, 36172]"
2,"[1803, 94810, 1, 29, 10, 2863, 4065, 4, 9699, ..."
3,"[1029, 8, 506, 43306, 11806]"
4,"[258, 2309, 1749, 16, 135, 6, 87]"


In [2]:
test_words = pd.read_csv('../features/test/seq_tokenized.csv')
test_words['seq_title_description'] = test_words['seq_title_description'].str.strip('[]').str.replace('\s+', '')
test_words['seq_title_description'] = test_words['seq_title_description'].apply(lambda x: x.split(','))
test_words.head()

,seq_title_description
0,"[160, 310, 3, 50289]"
1,"[8, 179, 8, 179, 104776, 78140, 1, 1375, 10, 1..."
2,"[9034, 8, 39, 329, 9034, 277, 853, 4508, 1040,..."
3,"[4280, 101825, 195, 8, 537, 16391, 1, 467, 4, ..."
4,"[132, 510, 351, 8, 3197, 448, 49, 27, 11, 185,..."


In [6]:
res = []
for t in test_words['seq_title_description']:
    if len(t) > 100:
        temp = t[:100]
    elif len(t) < 100:
        for i in range(100 - len(t)):
            t.append(0)
        temp = t
    else:
        temp = t
    temp = np.array([int(val) if val != '' else int(0) for val in temp])
    res.append(temp)
res = np.array(res)


In [7]:
x = torch.from_numpy(res)
x.shape

torch.Size([508438, 100])